In [ ]:
import json
from models.db_models import BasePromptTemplate, BasePromptTemplateV2
from llm_util import LLMProvider
from db_utils import DatabaseManager
from prompt_utils import getPromptFromTemplate


def extract_response_text(response) -> str:
    """Extract text from LLM response output"""
    if not response or not response.output:
        raise ValueError("Invalid response from LLM provider")
    
    for output in response.output:
        if output.type == "message" and output.content and len(output.content) > 0:
            return output.content[0].text
    
    return ""


def generate_character_data(llm_provider : LLMProvider, character_generation_pompt_template: BasePromptTemplateV2, db_manager: DatabaseManager, job_id: str) -> str:

    prompt = getPromptFromTemplate(character_generation_pompt_template).get("prompt", "")
    print(f"Character Generation Prompt: {prompt}")
    response_id = llm_provider.initiateResponse(prompt, model="gpt-5-2025-08-07", resoning_effort="medium")
    print(f"Submitted LLM request, response ID: {response_id}")

    response = llm_provider.getPooledResponse(response_id)
    response_text = extract_response_text(response)
    print(f"Response: {response_text}")

    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    
    # Store raw response text directly in the data field
    job.characterData = response_text
    db_manager.update_job_field(job_id, job)

    return job.characterData

def generate_plot(llm_provider: LLMProvider, plot_template: BasePromptTemplateV2, character_details: str, db_manager: DatabaseManager, job_id: str) -> str:
    """Step 1.2: Generate story plot"""
    # Replace {{characterData}} in the prompt template
    prompt = getPromptFromTemplate(plot_template).get("prompt", "")
    print(prompt)
    prompt = prompt.replace("{{CHARACTER_DETAILS}}", character_details)
    
    response_id = llm_provider.initiateResponse(prompt, model="gpt-5-2025-08-07", resoning_effort="medium")
    print(f"Submitted plot generation prompt: {prompt}")
    
    response = llm_provider.getPooledResponse(response_id)
    response_text = extract_response_text(response)
    print(f"Plot response: {response_text}")
    
    # Update job - store raw response text
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    job.plot = response_text
    db_manager.update_job_field(job_id, job)
    
    return job.plot


def generate_story_chain(llm_provider: LLMProvider, chain_template: BasePromptTemplate, plot: str, character_data: str, db_manager: DatabaseManager, job_id: str) -> str:
    """Step 1.3: Generate story chain"""
    prompt = chain_template.promptTemplate.replace("{{PLOT}}", plot).replace("{{CHARACTER_DATA}}", json.dumps(character_data))
    
    response_id = llm_provider.initiateResponse(prompt, model="gpt-5-2025-08-07", resoning_effort="medium")
    print(f"Submitted story chain request: \n {prompt}")
    
    response = llm_provider.getPooledResponse(response_id)
    response_text = extract_response_text(response)
    print(f"Story chain response: {response_text}")
    
    # Update job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    job.storyChain = response_text
    db_manager.update_job_field(job_id, job)
    
    return response_text


def generate_story_summary(llm_provider: LLMProvider, summary_template: BasePromptTemplate, plot: str, character_data: str, story_chain: str, db_manager: DatabaseManager, job_id: str) -> str:
    """Step 1.4: Generate story summary"""
    prompt = summary_template.promptTemplate.replace("{{STORY_PLOT}}", plot).replace("{{CHARACTER_DATA}}", json.dumps(character_data)).replace("{{STORY_CHAIN}}", story_chain)
    
    response_id = llm_provider.initiateResponse(prompt, model="gpt-5-2025-08-07", resoning_effort="medium")
    print(f"Submitted story summary request: \n {prompt}")
    
    response = llm_provider.getPooledResponse(response_id)
    summary_text = extract_response_text(response)
    print(f"Story summary length: {len(summary_text)} characters")
    
    # Update job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    job.storySummary = summary_text
    db_manager.update_job_field(job_id, job)
    
    return summary_text


def generate_first_draft(llm_provider: LLMProvider,
                         draft_template: BasePromptTemplate,
                         story_summary: str,
                            character_data: str,
                         db_manager: DatabaseManager,
                         job_id: str) -> str:
    """Step 1.5: Generate first draft"""
    prompt = draft_template.promptTemplate.replace("{{CHARACTER_DATA}}", json.dumps(character_data)).replace("{{STORY_SUMMARY}}", story_summary)
    
    response_id = llm_provider.initiateResponse(prompt, model="gpt-5-2025-08-07", resoning_effort="medium")
    print(f"Submitted first draft request: \n" f"{prompt}")
    
    response = llm_provider.getPooledResponse(response_id)
    draft_text = extract_response_text(response)
    
    # Update job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    job.firstDraft = draft_text
    db_manager.update_job_field(job_id, job)
    
    return draft_text


def enhance_climax(llm_provider: LLMProvider, climax_template: BasePromptTemplate, first_draft: str, db_manager: DatabaseManager, job_id: str) -> str:
    """Step 1.6: Enhance climax"""
    prompt = climax_template.promptTemplate.replace("{{FIRST_DRAFT}}", first_draft)
    
    response_id = llm_provider.initiateResponse(prompt)
    print(f"Submitted climax enhancement request, response ID: {response_id}")
    
    response = llm_provider.getPooledResponse(response_id)
    enhanced_text = extract_response_text(response)
    print(f"Climax enhanced story length: {len(enhanced_text)} characters")
    
    # Update job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    job.climaxEnhancedStory = enhanced_text
    db_manager.update_job_field(job_id, job)
    
    return enhanced_text


def align_with_storyverse(llm_provider: LLMProvider, alignment_template: BasePromptTemplate, climax_enhanced_story: str, db_manager: DatabaseManager, job_id: str) -> str:
    """Step 1.7: Align with storyverse"""
    prompt = alignment_template.promptTemplate.replace("{{CLIMAX_ENHANCED_STORY}}", climax_enhanced_story)
    
    response_id = llm_provider.initiateResponse(prompt)
    print(f"Submitted storyverse alignment request, response ID: {response_id}")
    
    response = llm_provider.getPooledResponse(response_id)
    final_text = extract_response_text(response)
    print(f"Final story length: {len(final_text)} characters")
    
    # Update job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    job.finalStory = final_text
    db_manager.update_job_field(job_id, job)
    
    return final_text

def get_job_and_handle_exceptions(db_manager: DatabaseManager, job_id: str):
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    return job

In [ ]:
from models.db_models import StoryverseMetaData, Job
import os
from dotenv import load_dotenv
from llm_util import OpenAiLLMProvider
from db_utils import DatabaseManager

if __name__ == "__main__":
    load_dotenv()
    
    # Initialize providers
    llm_provider = OpenAiLLMProvider()
    db_manager = DatabaseManager()
    
    # Get template
    story_verse = os.getenv('STORY_VERSE') or "SHERLOCK"
    meta_data = db_manager.get_meta_data(story_verse)

    ## do optional check of metadata
    if meta_data is None:
        raise ValueError(f"No metadata found for the given story verse: {story_verse}")
    
    job_id = ""
    
    # Create new job
    job = Job(
        storyVerse=story_verse,
        characterData="",  
        plot="",
        storyChain="",
        storySummary="",
        firstDraft="",
        climaxEnhancedStory="",
        finalStory=""
        )
    job_id = db_manager.create_job(job)
    
    print(f"Created job with ID: {job_id}")
    
    # STEP 1 - Generate Character Data
    print("=== STEP 1: Character Generation ===")
    characterData = generate_character_data(
        llm_provider,
        meta_data.characterGenearationPromptTemplate,
        db_manager,
        job_id)
    print(f"✓ Character data generated")
    
    # STEP 2 - Generate Plot
    print("=== STEP 2: Plot Generation ===")
    job = get_job_and_handle_exceptions(db_manager, job_id)
    plot = generate_plot(
        llm_provider,
        meta_data.plotGenerationPromptTemplate,
        job.characterData,
        db_manager,
        job_id)
    print(f"✓ Plot generated")
    
    # STEP 3 - Generate Story Chain
    print("=== STEP 3: Story Chain Generation ===")
    job = get_job_and_handle_exceptions(db_manager, job_id)
    story_chain = generate_story_chain(
        llm_provider,
        meta_data.storyChainGenerationPromptTemplate,
        job.plot,
        job.characterData,
        db_manager,
        job_id)
    print(f"✓ Story chain generated")
    
    # STEP 4 - Generate Story Summary
    print("=== STEP 4: Story Summary Generation ===")
    job = get_job_and_handle_exceptions(db_manager, job_id)
    story_summary = generate_story_summary(
        llm_provider,
        meta_data.storySummaryGenerationPromptTemplate,
        job.plot,
        job.characterData,
        job.storyChain,
        db_manager,
        job_id)
    print(f"✓ Story summary generated \n {story_summary})")
    
    # STEP 5 - Generate First Draft
    print("=== STEP 5: First Draft Generation ===")
    job = get_job_and_handle_exceptions(db_manager, "68d048b4902faf4c122d3f2e")
    first_draft = generate_first_draft(
        llm_provider,
        meta_data.fistDraftGenerationPromptTemplate,
        job.storySummary,
        job.characterData,
        db_manager,
        job_id)
    print(f"✓ First draft generated: \n {first_draft}")
    
    # # STEP 6 - Enhance Climax
    # print("=== STEP 6: Climax Enhancement ===")
    # climax_enhanced = enhance_climax(
    #     llm_provider,
    #     meta_data.climaxEnhancementPromptTemplate,
    #     first_draft,
    #     db_manager,
    #     job_id)
    # print(f"✓ Climax enhanced story generated ({len(climax_enhanced)} chars)")
    
    # # STEP 7 - Storyverse Alignment
    # print("=== STEP 7: Storyverse Alignment ===")
    # final_story = align_with_storyverse(
    #     llm_provider,
    #     meta_data.storyverseAlignmentPromptTemplate,
    #     climax_enhanced,
    #     db_manager,
    #     job_id)
    # print(f"✓ Final story generated ({len(final_story)} chars)")
    
    print(f"\n🎉 Story generation completed! Job ID: {job_id}")
    
    # # Close DB connection when done
    db_manager.close()